# Explainer Notebook

## 1. Motivation
What is your dataset?

Why did you choose this/these particular dataset(s)?

What was your goal for the end user’s experience?

Our dataset consists of animal pages on Wikipedia, where we used wescraping methods and the WikiData API to collect a dataset.
We choose this particular dataset because when looking at Wikipedia in general there is always a link to another page within a wikipedia page and then another link. This we thought would be interesting to look at in regards to the kingdom of animals. Because some groups of animals are more likely to mention each other in a wikipedia article. We first analyzed all of the approxomately 30k wikipedia pages and then narrowed down our network to only reptiles in order to narrow down the dataset.
Our goal with the analysis of our Wikipedia animal dataset was to show the user and viewer how contected these animals actually are on wikipedia.

## 2. Basic stats
Write about your choices in data cleaning and preprocessing

Write a short section that discusses the dataset stats (here you can recycle the work you did for Project Assignment A)


The data that will be used in the analysis, has been gathered by using the WikiData API using their query builder. Due to the Wikipedia pages not having a category called “Animals” that allows you to gather all the animal pages, we will use an alternative which is querying on the Wikidata pages that have a “Animal Diversity Website (ADW) taxon id”. The query to gather the data is:

In [ ]:
query = '''
    SELECT DISTINCT ?item ?itemLabel WHERE {
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE]". }
      {
        SELECT DISTINCT ?item WHERE {
          ?item p:P4024 ?statement0.
          ?statement0 (ps:P4024) _:anyValueP4024.
        }
      }
    }
    '''
query_reptile = '''
    SELECT DISTINCT ?item ?itemLabel WHERE {
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE]". }
      {
        SELECT DISTINCT ?item WHERE {
          ?item p:P4024 ?statement0.
          ?statement0 (ps:P4024) _:anyValueP4024.
          ?item p:P5473 ?statement1.
          ?statement1 (ps:P5473) _:anyValueP5473.
        }
      }
    }
    '''

In [ ]:
from tqdm import tqdm
import requests
import json
from bs4 import BeautifulSoup
import re
from get_links import links_on_page
import networkx as nx
import matplotlib.pyplot as plt
from netwulf import visualize
import pickle
import numpy as np

The first query just request all the pages where it has a Animal Diversity Website (ADW) taxon id, this query is made by their auto query builder.
The second query request all the pages where it has a Animal Diversity Website (ADW) taxon id aswell as a Reptile Database ID.

The get_wiki_links function takes n amount of wikidata ids, and in the query they are seperated with | as this allows us to query up to 50 ids at a time. We then loop over the response and check if it contains a link to an english wikipedia site.

In [ ]:
def get_wiki_links(item_ids):
    url = f"https://www.wikidata.org/w/api.php?action=wbgetentities&ids={'|'.join(item_ids)}&props=sitelinks/urls&format=json&sitefilter=enwiki"
    response = requests.get(url)
    data = json.loads(response.text)
    wiki_links = []
    for item_id in item_ids:
        wikipedia_url = data["entities"][item_id]["sitelinks"]
        if "enwiki" in wikipedia_url: # only save if the page has a reference to an english wikipage
            link = wikipedia_url["enwiki"]["url"]
            wiki_links.append(link)
    return wiki_links

For the actual quering, we start by getting all the wikidata pages that fit our query (query & query_reptile). We loop over 50 pages at a time, and get their wikipedia links with get_wiki_links, and write the results to a .txt file where each entry is seperated by a new line.

In [ ]:
url = 'https://query.wikidata.org/sparql'
r = requests.get(url, params = {'format': 'json', 'query': query_reptile})
data = r.json()

data = data["results"]["bindings"]
temp = []
for entries in tqdm(range(0,len(data),50)):
    if entries+50 < len(data):
        sub_list = [ids["itemLabel"]["value"] for ids in data[entries:(entries+50)]]
        temp = temp + get_wiki_links(sub_list)
    else:
        sub_list = [ids["itemLabel"]["value"] for ids in data[entries:(len(data))]]
        temp = temp + get_wiki_links(sub_list)
file = open('animal_links_reptile.txt','w')
for item in temp:
    file.write(item+"\n")
file.close()

Now for creating our nodes and edges. We open the .txt file from before, and prepare for the web scraping:

In [ ]:
edgelist_weights = {}
edgelist_weights_long = {}
#animal_name = "Elephant"
names = names_from_table() # Gets all the entries in the larger table on https://en.wikipedia.org/wiki/List_of_animal_names
names_long = {}
with open('data/animal_links_reptile.txt', 'r') as f:
    entries = f.read().splitlines()
for name in tqdm(entries):
    name_temp = name.split("/")[-1]
    names_long[name_temp] = 0 # Saving all the entries in the txt file in a dict, for a fast comparisons (ie. only make pairs
                                #  with animals and not wikipages for unrelated stuff)
attributes_dict = {}

For each wikipedia link in the .txt file, we get all the links on the page aswell as look for infoboxes using links_on_page. This function just uses the wikipedia api to get all the information on the page in json format, and then we find all href in the wikipedia page and save these.
Then we look for a infobox, since there are 2 ways wikipedia makes these, we have to check if infobox is none, as this indicates the infobox is the other type.
Then we go through the infobox (table) and extract predefined values we wanna use as attributes.
In case there is no infobox on the page, ie. infobox is never updated from having None as values, we discard this page as it is most likely a redirect page.

In [ ]:
def links_on_page(animal_name="Elephant"):
    url = "https://en.wikipedia.org/w/api.php?action=parse&page="+animal_name+"&format=json" # we can either put the title page or the URL version
                                                                                             # of the title, ex. Malayan softshell turtle = Malayan_softshell_turtle
    response = requests.get(url)
    html = json.loads(response.content.decode('utf-8'))['parse']['text']['*'] # Default way the result comes in
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.find_all('a', href=lambda href: href and href.startswith('/wiki/') and not href.endswith('.jpg') and not href.endswith('.png'))
    # ^ sometimes the links link to jpg and png file, sort them out
    result = []
    for link in links:
        href = link.get('href')
        title = link.get('title')
        text = link.text
        result.append(href)
    info = {"Class:": None, "Order:":None, "Superfamily:":None, "Family:":None,"Name:": None} # Default dict allows us to see if its redirect later
    infobox = soup.find('table', {'class': 'infobox biota biota-infobox'}) # One type of infoboxes wiki uses
    if infobox is None:
        infobox = soup.find('table', {'class': 'infobox biota'}) # Other type of infoboxes wiki uses
    if infobox:
        rows = infobox.find_all('tr') # Going through the rows in the infobox
        for row in rows:
            td = row.find_all('td')
            if td: # Checking if empty
                if td[0].text.strip() in ["Class:", "Order:", "Superfamily:", "Family:"]: # the info we want is stored in a row at a time
                                                                                          # it has two columns first being Class, Order...
                                                                                          # the other being the attributes we will save 
                    if td:
                        info[td[0].text.strip()] = td[1].text.strip() # Making the category the key, and the attribute value the value
        info["Name:"] = animal_name # Updating from default 

    return result, info

One of the graph we make is the one where we only make a edge a list of 224 animal names from https://en.wikipedia.org/wiki/List_of_animal_names, this wikipedia page contains two tables, one of them being one with the overall animal species (no subspecies).
Due to it being a wikipedia page, there is a lot of references and stuff we dont want in our scraping which is removed with re.sub, where we have given it some predefined things to remove.

In [ ]:
def names_from_table():
    url = "https://en.wikipedia.org/wiki/List_of_animal_names"
    
    response = requests.get(url)
    html_content = response.content
    
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Find table on the page
    table = soup.find_all('table',{"class":"wikitable"})[1]
    
    # Find all the rows in the table
    t_rows = table.find_all('tr')
    
    ths = t_rows[0].find_all('th')
    
    header = [th.text.replace("\n","") for th in ths]
    
    rows = []
    names = {}
    for tr in t_rows[1:]:
        tds = tr.find_all('td')
        if tds:
            links = tds[0].find_all('a')
            if links:
                row = [td.text.replace("\n","") for td in tds]
                rows.append(row)
                cleaned_row = re.sub(r'\(.*\)|Also see.*|\[\d+\]|See.*', '', row[0]) # The table is not clean, many unwanted formating we remove here
                for link in links:
                    link_href = link.get('href')
                    if link_href.startswith('/wiki/'):
                        names[link_href] = cleaned_row
                        break
    import pandas as pd
    df_animals = pd.DataFrame(rows, columns=header)

    return names

Now for the actual scraping of websites, we loop over all the entries from our .txt file. We check if its redirect by looking at if infobox is None (its default scenario) if not, we add it to our edgelist. One of them to be used for directed and the other for undirected graph.

In [ ]:
for name in tqdm(entries):
    temp_string = name.split("/")[-1]
    result, info = links_on_page(animal_name=temp_string)
    if info["Name:"] is not None: # A way to remove wiki redirects from the final result, as redirects dont have the infoboxes we want
        attributes_dict[info["Name:"]] = info # Making nested dicts to quickly get attributes later
        for entry in result:
            if entry in names: # making one graph where we only make edges to the table from https://en.wikipedia.org/wiki/List_of_animal_names
                pair = ("/wiki/"+temp_string,entry) # Making pairs to compare for the dict
                pair_inverted = (entry,"/wiki/"+temp_string)
                if pair in edgelist_weights:
                    edgelist_weights[pair] += 1 # If the pair is already in the dict, the weight is increased
                elif pair_inverted in edgelist_weights:
                    edgelist_weights[pair_inverted] += 1 # If the inverted pair is already in the dict, the weight is increased
                else:
                    edgelist_weights[pair] = 1 # If the pair is not in the dict, the weight is 1
            if entry.split("/")[-1] in names_long: # making other graph where we make edges between entries from the .txt file
                pair = (temp_string, entry.split("/")[-1]) # Only taking the last part of the URL (the URL title of the page)
                if pair in edgelist_weights_long:
                    edgelist_weights_long[pair] += 1 # If the pair is already in the dict, the weight is increased
                else:
                    edgelist_weights_long[pair] = 1 # If the pair is not in the dict, the weight is 1


In order to avoid doing this over and over we dump the results as pickle files

In [ ]:
with open('data/data_plain_reptile_test.pickle', 'wb') as fp:
    pickle.dump(edgelist_weights, fp, protocol=pickle.HIGHEST_PROTOCOL)
with open('data/data_plain_long_reptile_test.pickle', 'wb') as fp:
    pickle.dump(edgelist_weights_long, fp, protocol=pickle.HIGHEST_PROTOCOL)
with open('data/Reptile_attributes.pickle', 'wb') as fp:
    pickle.dump(attributes_dict, fp, protocol=pickle.HIGHEST_PROTOCOL)


To visulize them, they are first loaded and then we loop over all the entries and add them to an edgelist, and construct the graph with this edgelist.
We then add all our attributes to the graph, and for sanity check we remove nodes than dont have attributes saved in our .pickle file.

In [ ]:
with open('data/all_animal_to_all_animal.pickle', 'rb') as handle:
    b = pickle.load(handle)
with open('data/animal_attributes.pickle', 'rb') as handle:
    c = pickle.load(handle)

values = list(b.values())
plt.hist(values, bins=np.arange(max(values))-0.5, edgecolor='black')
plt.yscale("log")
plt.xticks(range(1, max(values) + 1))
plt.xlabel('Number of references')
plt.ylabel('Frequency')
plt.show()

edgelist = [None]*len(b)
for i,items in enumerate(b):
    edgelist[i] = (items[0].replace("/wiki/", ""),items[1].replace("/wiki/", ""),int(b[items]))
G = nx.DiGraph()

G.add_weighted_edges_from(edgelist)
print(G)

to_remove =[]
for Names in tqdm(G.nodes):
    if Names in c:
        G.nodes[Names]['Class'], G.nodes[Names]['Order'], G.nodes[Names]['Superfamily'], G.nodes[Names]['Family'], _ = c[Names].values()
    else:
        to_remove.append(Names) # Some nodes get added to graph even though they are redirects, the cause is known but no good way to handle it
for names in to_remove:
    G.remove_node(names)
print(G)
network, config = visualize(G)

degree = []
for node in G.nodes():
    degree.append(G.degree(node))
plt.hist(degree,bins=np.arange(max(values))-0.5, edgecolor='black')
plt.title("Degree distribution")
plt.xlabel('Number of degrees')
plt.ylabel('Frequency')
plt.show()

## 3. Tools, theory and analysis. Describe the process of theory to insight
Describe which network science tools and data analysis strategies you’ve used, how those network science measures work, and why the tools you’ve chosen are right for the problem you’re solving.

Talk about how you’ve worked with text, including regular expressions, unicode, etc.

How did you use the tools to understand your dataset?


### Network theory

Fraction of edges

Assortivity

Modularity

$M=\sum_{c=1}^{n_c}\left[\frac{L_c}{L}-(\frac{k_c}{2L})^2\right]$ 

##### Louvain
The Louvain algorithm is a community detection algorithm used to identify the communities or groups within a network.

The algorithm works by optimizing a modularity function that measures the quality of the community structure. The modularity function quantifies the extent to which the number of edges within communities is higher than the expected number in a random network with the same degree sequence.

Opening the stored data with two helper functions, clean_family cleans the attribute values from wikipedia as these can contain symbols and extra stuff we are not interested in. The add_attr function adds the attributes to the network.

The banned set, is for cleaning up the nodes for the "cleaned" network described in https://natasha0301.github.io/Project-website/network-analysis/

Please note some of the functions used below, are either directly from previous assignments or modified to fit the problem at hand.

In [ ]:
import networkx as nx
import pickle
import random
import community
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import colorsys
from netwulf import visualize
import pandas as pd


def clean_family(string):
    if string is None:
        return None
    string = string.replace("†","").replace(" (","")
    count = 0
    for i,chara in enumerate(string):
        if chara.isupper():
            count += 1
            if count > 1:
                return string[:i]
    return string

def add_attr(Graph,attr_dict):
    to_remove =[]
    for Names in Graph.nodes:
        if Names in attr_dict:
            Graph.nodes[Names]['Class'], Graph.nodes[Names]['Order'], Graph.nodes[Names]['Superfamily'], Graph.nodes[Names]['Family'], _ = attr_dict[Names].values()
            Graph.nodes[Names]['Class'] = clean_family(Graph.nodes[Names]['Class']) if Graph.nodes[Names]['Class'] is not None else None
        else:
            to_remove.append(Names) # Some nodes get added to graph even though they are redirects, the cause is known but no good way to handle it
    for names in to_remove:
        Graph.remove_node(names)
    return Graph

# Load Network
with open('data/all_animal_to_all_animal.pickle', 'rb') as handle:
    b = pickle.load(handle)
with open('data/animal_attributes.pickle', 'rb') as handle:
    c = pickle.load(handle)
banned_set = {"Animal","Reptile","Arthropod","Chordate","Bird","Rodent","Insect"}
for items in c:
    banned_set.add(clean_family(c[items]["Class:"]))
    banned_set.add(clean_family(c[items]["Order:"]))
    banned_set.add(clean_family(c[items]["Superfamily:"]))
    banned_set.add(clean_family(c[items]["Family:"]))
#edgelist = [None]*len(b)
banned_set = {}
edgelist = []
for i,items in enumerate(b):
    if items[0] not in banned_set and items[1] not in banned_set:
        edgelist.append([items[0].replace("/wiki/", ""),items[1].replace("/wiki/", ""),int(b[items])])
G_reptile = nx.Graph()


G_reptile.add_weighted_edges_from(edgelist)
print(G_reptile)
G_reptile_attr = add_attr(G_reptile,c)

Below are the functions to calculate the fraction of edges in the same class or Family. The functions are not smart, ie. have to manully change between Class and Family.

In [ ]:
def same_class(G):
    same_class_fractions = []
    for node in G.nodes():
        same_class_fractions = 0
        total_neighbors = 0
        
        for neighbor in G.neighbors(node):
            if G.nodes[neighbor]["Class"] == G.nodes[node]["Class"]:
                same_class_fractions += 1
            total_neighbors += 1
        
        if total_neighbors > 0:
            same_field_fraction = same_class_fractions / total_neighbors
        else:
            same_field_fraction = 0
        same_class_fractions.append(same_field_fraction)
    return same_class_fractions

def same_class_rand(Graph):
    shuffled_G = Graph.copy()
    Family = [Graph.nodes[node]["Class"] for node in Graph.nodes()]
    random.shuffle(Family)

    for i, node in enumerate(shuffled_G.nodes()):
        shuffled_G.nodes[node]["Class"] = Family[i]
    return shuffled_G

def same_class_rand_n(Graph, n = 250):
    results = []
    Family = [Graph.nodes[node]["Class"] for node in Graph.nodes()]
    for i in range(n):
        temp = same_class_rand(Graph)
        results.append(np.mean(same_class(temp)))
    plt.hist(results,label = "Random", bins = 20)
    plt.xlabel("Fraction")
    plt.ylabel("Frequency")
    plt.title("Fraction of edges in same Class")
    plt.show()

Due to the size of the network we are not able to do a random shuffles between the edges, in a timely manner (network has >500,000 edges).

In [ ]:
# Same class fractions
same_class_fractions = same_class(G_reptile_attr)
print(f"The average fraction is: {np.mean(same_class_fractions):.3f}")

# Same class fractions for random graph
same_class_fractions_rand = same_class_rand(G_reptile_attr)
print(f"The average fraction is: {np.mean(same_class(same_class_fractions_rand)):.3f}")

# Same class fractions
#same_family_rand_n(G_reptile_attr, n = 250)


For the assortativity coefficient, we reuse function from last assignment. This metric is used to tell  how likely similar nodes are to connect to other similar nodes over dissimilar nodes. In other words it reflects the degree of homophily or heterophily in the network. 

In [ ]:
def assortative_matrix(Graph,unique_labels):
    matrix = np.zeros((len(unique_labels),len(unique_labels)))

    values = nx.get_node_attributes(Graph, "Class").values()
    num_values = len(values)

    for start, end in Graph.edges(): # Looping over all edges in graph (since its undirected its not really start and stop)
        x = Graph.nodes[start]["Class"] # Getting the start point of the edge 
        y = Graph.nodes[end]["Class"] # Getting the end point of the edge 
        if x in unique_labels: 
            x = unique_labels[x]
        else:
            x = unique_labels[None] # in case x is nan
            
        if y in unique_labels:
            y = unique_labels[y]
        else:
            y = unique_labels[None] # in case y is nan
        matrix[x, y] += 1 
        
    num_edges = len(Graph.edges())
    matrix /= num_edges # averaging the occurence with the total edges

    trace = np.trace(matrix) # trace of the matrix, the sum of the diagonal entries
    mix_matrix = np.sum(np.matmul(matrix, matrix))

    r1 = (trace-mix_matrix)/(1-mix_matrix) # Eq. 2

    return r1

In [ ]:
# Assortativity coefficient 
labels = np.unique([G_reptile_attr.nodes[node]["Class"] for node in G_reptile_attr.nodes() if G_reptile_attr.nodes[node]["Class"] is not None])
unique_labels = {name: i for i,name in enumerate(labels)}
unique_labels[None] = len(unique_labels)
r1 = assortative_matrix(G_reptile_attr,unique_labels)
print(f"Assortativity coefficient: {r1:.3f}")

The compute modularity function is unchanged from last assigment.

In [ ]:
def compute_modularity(Graph,partitioning):
    L = Graph.number_of_edges()
    M = 0
    communities = set(partitioning.values())

    for community in communities:
        nodes_in_community = [node for node, community_id in partitioning.items() if community_id == community]
        subgraph = Graph.subgraph(nodes_in_community)
        k_c = sum(dict(subgraph.degree()).values())
        L_c = subgraph.number_of_edges()
        M += L_c / L - (k_c / (2 * L)) ** 2

    return M

In [ ]:
# Modularity Family split of Reptile Graph
Family_split = nx.get_node_attributes(G_reptile_attr, "Class")

#Then we compute the modularity of the partitioning by using the function above
modularity = compute_modularity(G_reptile_attr, Family_split)

print(f"The modularity of the family split partitioning is {modularity:.3f}")

The modularity is used to see the degree of clustering and how well the nodes can be organized into distinct communities. This measure, measures the identification of meaningful substructures within the network

Due to the size of the network it is not feasible to construct new random graphs, to determine if they are statistically different from zero, however the function to do this for smaller networks is:

In [ ]:
def double_edge_swap(GG,N):
    """
    Example usage: G_new = double_edge_swap(G_reptile,len(list(G_reptile.edges())))
    """
    i = 0
    G_copy = GG.copy()
    while(i<2*N):
        edges = list(G_copy.edges()) # update edges after adding and removal
        (u, v), (x, y) = random.sample(edges, 2) # picking two random edges
        if (u != v) and (v != x) and (u, y) not in G_copy.edges() and (x, v) not in G_copy.edges(): # checking conditions
            G_copy.add_edge(u, y)
            G_copy.add_edge(x, v)
            G_copy.remove_edge(u, v)
            G_copy.remove_edge(x, y)
            i+=1
    return G_copy


We will now use the louvain community detection algoritm

In [ ]:
#Louvain algorithm
partition = community.best_partition(G_reptile,random_state=42) # dictionary, keys are the nodes and values are communities for each node

# modularity of partition
modularity = community.modularity(partition, G_reptile) 

size = []
for community_ in set(partition.values()):
    temp = []
    for part in partition:
        if partition[part] == community_:
            temp.append(part)
    size.append(len(temp))

print("Number of communities:", len(set(partition.values())))
print("Community sizes:", sorted(size,reverse=True))
print(f"Modularity: {modularity:.2f}")

To determine the performance we will find the accuracy aswell as construct a confusion matrix. We start by getting the partion done by Louvain, and then mapping the pred labes and true labels, as the nodes are in the same order.

We then look for where the predicted is the same as the ground truth, and use np.mean to get the accuracy.

For the confusion matrix we construct the empty array and populate it by counting the number of times (true,pred) pair occurs.

In [ ]:
louvain_labels = list(partition.values())

#ground truth community labels as a list
gt_labels = [G_reptile_attr.nodes[node_id]["Class"] for node_id in G_reptile_attr.nodes()]
gt_labels = [labels if labels is not None else "None" for labels in gt_labels] # convert None to string(None)

#mapping between pred labels and ground truth labels
label_mapping = {}
for louvain_label, gt_label in zip(louvain_labels, gt_labels):
    if louvain_label not in label_mapping:
        label_mapping[louvain_label] = gt_label


louvain_labels_mapped = [label_mapping[label] for label in louvain_labels] #convert pred labels to ground truth labels

accuracy = np.mean([1 if louvain == gt else 0 for louvain, gt in zip(louvain_labels_mapped,gt_labels)])
print(f"Accuracy of Louvain: {accuracy:.3f}")

labels = sorted(set(gt_labels + louvain_labels_mapped)) #Create a sorted list of unique labels
num_classes = len(labels)

cm = np.zeros((num_classes, num_classes), dtype=int)

# Count the number of occurrences of each (true, pred) label pair
for true, pred in zip(gt_labels, louvain_labels_mapped):
    cm[labels.index(true)][labels.index(pred)] += 1

confusion_matrix_df = pd.DataFrame(cm, index=unique_labels, columns=range(num_classes))
print(confusion_matrix_df)

To visualise the graph with different splits we create n unique colors and add a color attribute to our graph. The first uses Louvain splits and the second uses attribute splits.

In [ ]:
num_communities = len(set(partition.values()))
hue_start = 0.0
saturation = 0.8
value = 0.8

colors = []
for i in range(num_communities):
    hue = hue_start + (i / num_communities)
    r, g, b = colorsys.hsv_to_rgb(hue, saturation, value)
    color_hex = "#{:02x}{:02x}{:02x}".format(int(r * 255), int(g * 255), int(b * 255))
    colors.append(color_hex)

community_numb = list(partition.values())
for i, n in enumerate(G_reptile_attr.nodes()):
    G_reptile.nodes[n]['color'] = colors[community_numb[i]]
network, config = visualize(G_reptile)

In [ ]:
colors = []
for i in range(len(unique_labels)):
    hue = hue_start + (i / len(unique_labels))
    r, g, b = colorsys.hsv_to_rgb(hue, saturation, value)
    color_hex = "#{:02x}{:02x}{:02x}".format(int(r * 255), int(g * 255), int(b * 255))
    colors.append(color_hex)

print(f"Amount of unique familys in dataset: {len(unique_labels):.0f}")
for i, n in enumerate(G_reptile_attr.nodes()):
    G_reptile.nodes[n]['color'] = colors[unique_labels[G_reptile_attr.nodes[n]["Class"]]]

network, config = visualize(G_reptile)

### Text theory 

TF-IDF 

## 4. Discussion. Think critically about your creation
What went well?

What is still missing? 

What could be improved? Why?